## **Federated Learning for attack detection: 5 nodes sharing gradients: MEDIAN**

IDs from this file = **idMEDxy** (x = 0 if experiment with dataset, x = 1 if epochs & iterations, y being integer equal or greater than 0)

In this file, experiments with different datasets, and number of epochs & iterations are done. The experiments are divided into sections, based on the elements being changed. 

### Static elements for all experiments (execute first)

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [5]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')
test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_2808/3457440085.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_2808/3457440085.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')


In [6]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)
xcomplete, ycomplete = preprocessing(test_complete)

### Experiments with datasets

#### Dataset Filt5A

In [7]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5A-Part5.csv', low_memory=False)

In [8]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [9]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idMED00.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [10]:
def aggregate(grad_list):
    
    # Compute the median gradient for each layer
    med_grad = [
        tf.numpy_function(lambda grads: np.median(grads, axis=0), [tf.stack(layer_grads)], tf.float32)
        for layer_grads in zip(*grad_list)
    ]
    return med_grad

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idMED00.hdf5')

Epoch 1/5
31/31 [==============================] - 13s 409ms/step - loss: 0.3975 - accuracy: 0.9656 - val_loss: 1.6841 - val_accuracy: 0.6197
Epoch 2/5
31/31 [==============================] - 11s 342ms/step - loss: 0.0641 - accuracy: 0.9914 - val_loss: 0.9172 - val_accuracy: 0.6200
Epoch 3/5
31/31 [==============================] - 8s 251ms/step - loss: 0.0307 - accuracy: 0.9918 - val_loss: 0.8356 - val_accuracy: 0.6233
Epoch 4/5
31/31 [==============================] - 12s 392ms/step - loss: 0.0270 - accuracy: 0.9922 - val_loss: 0.6461 - val_accuracy: 0.6453
Epoch 5/5
31/31 [==============================] - 11s 359ms/step - loss: 0.0257 - accuracy: 0.9921 - val_loss: 0.5399 - val_accuracy: 0.6733
Epoch 1/5
31/31 [==============================] - 13s 404ms/step - loss: 0.0435 - accuracy: 0.9900 - val_loss: 0.6338 - val_accuracy: 0.6925
Epoch 2/5
31/31 [==============================] - 11s 354ms/step - loss: 0.0270 - accuracy: 0.9910 - val_loss: 0.3758 - val_accuracy: 0.7880
Epoch 3

In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.04021567478775978, 0.2003970593214035, 0.06247953698039055], [0.08260253071784973, 0.43851250410079956, 0.1389634609222412], [0.08335065841674805, 0.4406733512878418, 0.15186883509159088]]
Accuracy for iterations:  [[0.9814484715461731, 0.9205660223960876, 0.9726998209953308], [0.9688568115234375, 0.7853302359580994, 0.9418911933898926], [0.9685865640640259, 0.7905797958374023, 0.9380689263343811]]
F1 for iterations:  [[0.9814405848988382, 0.9197056210882352, 0.9726930520938676], [0.9687899127092552, 0.7695675686308082, 0.9417341465116771], [0.9685108164478585, 0.7759273251167887, 0.9378625699018214]]
Precision for iterations:  [[0.9815186781177139, 0.9254324948106357, 0.9731684551937836], [0.970011102049446, 0.8355941927317043, 0.9467069664943243], [0.9699706538488233, 0.8385065526992616, 0.9439649392456899]]
Recall for iterations:  [[0.9814484793596073, 0.920566010317518, 0.9726998165572753], [0.9688568173186479, 0.7853302332340333, 0.9418912038167246], [0.9

#### Dataset Filt5B

In [16]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5B-Part5.csv', low_memory=False)

In [17]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idMED01.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idMED01.hdf5')

Epoch 1/5
31/31 [==============================] - 17s 562ms/step - loss: 0.3846 - accuracy: 0.9539 - val_loss: 2.0922 - val_accuracy: 0.5504
Epoch 2/5
31/31 [==============================] - 13s 420ms/step - loss: 0.0644 - accuracy: 0.9912 - val_loss: 1.4158 - val_accuracy: 0.5526
Epoch 3/5
31/31 [==============================] - 11s 359ms/step - loss: 0.0297 - accuracy: 0.9917 - val_loss: 0.7306 - val_accuracy: 0.5773
Epoch 4/5
31/31 [==============================] - 8s 258ms/step - loss: 0.0261 - accuracy: 0.9921 - val_loss: 0.6551 - val_accuracy: 0.6053
Epoch 5/5
31/31 [==============================] - 11s 366ms/step - loss: 0.0247 - accuracy: 0.9920 - val_loss: 0.5809 - val_accuracy: 0.6394
Epoch 1/5
31/31 [==============================] - 15s 494ms/step - loss: 0.0342 - accuracy: 0.9902 - val_loss: 0.4173 - val_accuracy: 0.7676
Epoch 2/5
31/31 [==============================] - 10s 313ms/step - loss: 0.0249 - accuracy: 0.9913 - val_loss: 0.3211 - val_accuracy: 0.8073
Epoch 3

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.14167417585849762, 0.2995983958244324, 0.20479938387870789], [0.08905424177646637, 0.25220876932144165, 0.1466495245695114], [0.09121678024530411, 0.254693865776062, 0.15116024017333984]]
Accuracy for iterations:  [[0.9240118265151978, 0.8116871118545532, 0.8911201357841492], [0.9534313082695007, 0.8691055774688721, 0.9211475253105164], [0.9524891376495361, 0.8683063387870789, 0.9217055439949036]]
F1 for iterations:  [[0.9232854151269415, 0.8010585992597671, 0.8899009860283438], [0.9532346007403208, 0.8653498725888372, 0.920712350375337], [0.9522853829097507, 0.8645479830216747, 0.9212841612252541]]
Precision for iterations:  [[0.9326049342184513, 0.849999150947512, 0.9092451437869306], [0.9564657771770103, 0.8870928244969982, 0.9306001356404968], [0.9555799542266943, 0.8860697097195477, 0.9309317373909491]]
Recall for iterations:  [[0.9240118028572263, 0.8116871321659522, 0.8911201166221429], [0.9534313009436443, 0.8691055729128824, 0.9211475284057223], [0.95

#### Dataset Filt5C

In [24]:
# Load training datasets and testing datasets 
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part1.csv', low_memory=False)
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part2.csv', low_memory=False)
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part3.csv', low_memory=False)
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part4.csv', low_memory=False)
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Filt5C-Part5.csv', low_memory=False)

In [25]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idMED02.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idMED02.hdf5')

Epoch 1/5
16/16 [==============================] - 6s 371ms/step - loss: 0.5397 - accuracy: 0.7491 - val_loss: 1.7106 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 5s 300ms/step - loss: 0.2074 - accuracy: 0.9489 - val_loss: 3.6385 - val_accuracy: 0.1299
Epoch 3/5
16/16 [==============================] - 4s 253ms/step - loss: 0.0849 - accuracy: 0.9893 - val_loss: 3.9016 - val_accuracy: 0.1339
Epoch 4/5
16/16 [==============================] - 3s 207ms/step - loss: 0.0507 - accuracy: 0.9894 - val_loss: 1.7949 - val_accuracy: 0.1355
Epoch 5/5
16/16 [==============================] - 4s 250ms/step - loss: 0.0352 - accuracy: 0.9899 - val_loss: 1.3799 - val_accuracy: 0.1771
Epoch 1/5
14/14 [==============================] - 5s 387ms/step - loss: 0.0225 - accuracy: 0.9949 - val_loss: 0.8095 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 4s 283ms/step - loss: 0.0200 - accuracy: 0.9950 - val_loss: 0.5206 - val_accuracy: 0.7311
Epoch 3/5

In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.07833056151866913, 0.16251912713050842, 0.08799697458744049], [0.058851320296525955, 0.09940706938505173, 0.05509649217128754], [0.060660067945718765, 0.0856747254729271, 0.05493783578276634]]
Accuracy for iterations:  [[0.9878976941108704, 0.9515185356140137, 0.9812790751457214], [0.9893438220024109, 0.979419469833374, 0.9900745749473572], [0.9887887239456177, 0.9837789535522461, 0.9900466799736023]]
F1 for iterations:  [[0.9879017615640726, 0.951333855590243, 0.9812790133480048], [0.989349069254417, 0.9794558062330426, 0.990073768534064], [0.9887945470246919, 0.9838066189632456, 0.9900456965845631]]
Precision for iterations:  [[0.9879915605163647, 0.9524089898434744, 0.9812856042865771], [0.9895683243562778, 0.9800396702321673, 0.9902316018026237], [0.9890446953678572, 0.9843397518722772, 0.9902371400138832]]
Recall for iterations:  [[0.9878976890940431, 0.9515185642665117, 0.9812790770668695], [0.9893438312541997, 0.979419457967013, 0.9900745628413394], [0.